In [ ]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import paramiko

This is specifically for analyzing 1-meter photometry data, which I have stored in a specific place.

In [ ]:
fullFilePath = '/mnt/mrawls/1m_obs/'
hostname = 'seismo'
username = 'mrawls'  # password is magically saved in SSH file-land
# Not sure if having a config file actually helps or not
#config_file = '/Users/Meredith/.ssh/config'
#ssh_config = paramiko.SSHConfig()
#ssh_config.parse(open(config_file))
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.load_system_host_keys()
try:        # maybe we're on campus
    client.connect(hostname, 22, username)
    print('success! connected')
except:  # maybe we're not on campus
    print('connecting to astronomy first')
    client.connect('astronomy.nmsu.edu', 22, username)
    proxy_command = 'ssh %s@astronomy.nmsu.edu nc %s %s' % (username, hostname+'.nmsu.edu', 22)
    proxy = paramiko.ProxyCommand(proxy_command)
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(hostname, 22, username, sock=proxy)
    print('success! connected')

In [ ]:
sftp = client.open_sftp()
listOfAllFiles = sftp.listdir(fullFilePath)  # get all files and directories
listOfDirs = []
for item in listOfAllFiles:
    # get only files/directories that are dates (eg '140101')
    # (we assume these are all directories, and none are files)
    #if item[0:2] == ('14' or '15'):
    if item[0:2] == ('15'):
        listOfDirs.append(fullFilePath + str(item) + '/')
print('First directory is {0}'.format(listOfDirs[0]))
print('Last directory is {0}'.format(listOfDirs[-1]))

In [ ]:
#listOfFitsFiles = []
fout = open('imginventory_out.txt', 'w')
for directory in listOfDirs:
    print('next is {0} directory'.format(directory))
    sftp = client.open_sftp()
    listOfFilesInDir = sftp.listdir(directory)
    print('successfully accessed {0} directory'.format(directory))
    for filename in listOfFilesInDir:
        if filename[-4:] == ('fits' or 'FITS'):
            fullfilepath = directory+filename
            try:
                hdu = fits.open(sftp.file(fullfilepath))
                dateobs = (hdu[0].header['date-obs'])
                RA = (hdu[0].header['RA'])
                Dec = (hdu[0].header['Dec'])
                filtname = (hdu[0].header['filtname'][-10:])
                print(dateobs, RA, Dec, filtname, file=fout)
                #print(hdu[0].header['date-obs'], hdu[0].header['RA'], hdu[0].header['filtname'][-9])
            except:
                continue

#    except: # if we've exhausted the SSH volume, disconnect and reconnect
#        try:
#            client.close()
#            client.connect(hostname, 22, username)
#            print('reconnected!')
#            sftp = client.open_sftp()
#            listOfFilesInDir = sftp.listdir(dir)
#            print('successfully accessed {0} directory'.format(dir))
#            for file in listOfFilesInDir:
#                if file[-4:] == ('fits' or 'FITS'):
#                    fullfilepath = dir+file
#                    try:
#                        hdu = fits.open(sftp.file(fullfilepath))
#                        print(hdu[0].header['date-obs'], hdu[0].header['RA'], hdu[0].header['filtname'][-9])
#                    except:
#                        continue
#        except: # if reconnecting doesn't work, we're going to have a bad time
#            print('sorry, skipping {0} directory'.format(dir))
#            continue
